# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

In [2]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from shuscribe.schemas.llm import MessageRole

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [3]:
# Streaming response
from typing import Type
from pydantic import BaseModel
from shuscribe.services.llm.streaming import StreamStatus


async def stream(provider_name: str, messages: list[Message], response_schema: Type[BaseModel] | None = None):
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(
            temperature=0.7,
            response_schema=response_schema if response_schema else None
        )
        
        print(f"{TEST_MODELS[provider_name]}:")

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=TEST_MODELS[provider_name],
            config=config
        ):
            print(chunk.text, end="", flush=True)

    if chunk:
        if chunk.status == StreamStatus.COMPLETE:
            return chunk.accumulated_text


In [4]:
import yaml

from shuscribe.schemas.pipeline import Chapter, StoryMetadata

with open("../backend/tests/resources/pokemon_amber/_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)
    STORY_METADATA = StoryMetadata(
        title=meta.get('story_title'),
        description=meta.get('story_description'),
        genres=meta.get('genres'),
        additional_tags=meta.get('additional_tags')
    )

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(f"../backend/tests/resources/pokemon_amber/{chapter}", "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            continue


# Summarization Pipeline

## Chapter Summary

In [5]:
# Set Provider Name
PROVIDER_NAME = "gemini"
CHAPTER_INDEX = 0

In [6]:
from shuscribe.schemas.wikigen.summary import ChapterSummary
from shuscribe.services.llm.prompts import templates

templates.chapter.summary.reload()
messages: list[Message] = templates.chapter.summary.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    story_metadata=STORY_METADATA,
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter.")
)

# print(messages[-1].content)

summary_response = await run_async(stream(PROVIDER_NAME, messages))
chapter_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, summary_response)

gemini-2.0-flash-001:
<|STARTOFSUMMARY|>
# Chapter Summary

### Real-World Intro: A Gamer's Dilemma
*   The protagonist, a dedicated Pokémon player, is on the verge of losing a crucial battle in Pokémon Mansion on an emulator.
    *   Her Gyarados is at critical health against a Vulpix.
    *   She's reluctant to switch out her Pokémon due to the risk of losing them permanently, given the perma-death rules she's playing with.
*   A Discord notification alerts her to a Shadow Mewtwo raid happening at the library gym on campus.
    *   She saves her game state and decides to attend the raid.
*   While rushing to the raid, she gets hit by a truck.
    *   Her last thought is about her Gyarados.

### Rebirth: An Amber Awakening
*   The protagonist wakes up in a tank filled with amber fluid.
    *   She feels disconnected from her body.
    *   She can hear muffled sounds and feel vibrations.
*   The tank breaks, and she's caught by a middle-aged man, who is overjoyed.
    *   The man calls

Database module not implemented. Skipping save.


## Extract Entities

In [7]:
chapter_summary.to_prompt()

'<Content>\n\n# Chapter Summary\n\n### Real-World Intro: A Gamer\'s Dilemma\n*   The protagonist, a dedicated Pokémon player, is on the verge of losing a crucial battle in Pokémon Mansion on an emulator.\n    *   Her Gyarados is at critical health against a Vulpix.\n    *   She\'s reluctant to switch out her Pokémon due to the risk of losing them permanently, given the perma-death rules she\'s playing with.\n*   A Discord notification alerts her to a Shadow Mewtwo raid happening at the library gym on campus.\n    *   She saves her game state and decides to attend the raid.\n*   While rushing to the raid, she gets hit by a truck.\n    *   Her last thought is about her Gyarados.\n\n### Rebirth: An Amber Awakening\n*   The protagonist wakes up in a tank filled with amber fluid.\n    *   She feels disconnected from her body.\n    *   She can hear muffled sounds and feel vibrations.\n*   The tank breaks, and she\'s caught by a middle-aged man, who is overjoyed.\n    *   The man calls her "A

In [11]:


templates.entity.extract.reload()
messages: list[Message] = templates.entity.extract.format( 
    # current_chapter=CHAPTERS[CHAPTER_INDEX],
    current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter."),
    story_metadata=STORY_METADATA,
    chapter_summary=chapter_summary,
)

print(messages[-1].content)

# streaming_response = await run_async(stream(PROVIDER_NAME, messages))

# Task
Identify and extract all significant narrative entities from this chapter.

# Story Metadata
Title: Pokemon: Ambertwo [Pokemon Fanfic/Isekai]
Genres: ['Drama', 'Action', 'Adventure', 'Fantasy']
Additional Tags: ['Reincarnation', 'Portal Fantasy/Isekai', 'Fan Fiction', 'Female Lead', 'Genetically Engineered']
Description: |
Pokemon fan gets isekai'd to the Pokemon world as a little girl.

Join Dr. Fuji's apparently successful clone, as she explores this mishmash of Pokemon media.

# Narrative Context

## Current Chapter

### [1] Chapter 1
<Content>
This is a test chapter.
</Content>

## Current Chapter Summary
<Content>

# Chapter Summary

### Real-World Intro: A Gamer's Dilemma
*   The protagonist, a dedicated Pokémon player, is on the verge of losing a crucial battle in Pokémon Mansion on an emulator.
    *   Her Gyarados is at critical health against a Vulpix.
    *   She's reluctant to switch out her Pokémon due to the risk of losing them permanently, given the perma-death ru